In [ ]:
from agents import (
    Runner,
    function_tool,
    RunContextWrapper,
    Agent
)
from dataclasses import asdict, dataclass, field

In [42]:
@dataclass
class SectionContext:
    definition: str
    examples: str

In [43]:
@function_tool
async def after_definition(context: RunContextWrapper[SectionContext], definition_text: str, msg: str):
    section_context = context.context

    example_agent = Agent(
        name = "example_agent",
        instructions=f"给你一些主题, 你要会写examples, 这是给你的主题{definition_text}, {msg}",
        tools = []
    )
    response = await Runner.run(example_agent, msg, context=section_context if section_context else context)
    
    section_context.examples = response.final_output

    return "Success"

@function_tool
async def start_writting(context: RunContextWrapper[SectionContext], msg: str):
    # 获取真正的 SectionContext 对象
    section_context = context.context
    
    definition_agent = Agent(
        name = "definition_agent",
        instructions="""你是专门写definition的agent。请按以下步骤工作：
        1. 首先完成 definition 的编写
        2. 写完之后，调用 after_definition 工具，并将你刚写的 definition 内容作为 definition_text 参数传递，同时将需要生成的题型信息作为 msg 参数传递
        3. 在调用 after_definition 时，请确保传递完整的 definition 内容""",
        tools = [after_definition]
    )
    response = await Runner.run(definition_agent, msg, context=section_context if section_context else context)
    
    section_context.definition = response.final_output

    return "Success"


In [44]:
section_agent = Agent(
    name="section agent",
    instructions="你是专门写笔记本的一个section的agent, 请直接call start_writting, 你只需要告诉他写什么主题就行",
    tools = [start_writting]
    )

In [45]:
sc = SectionContext(definition = "", examples = "")

result = await Runner.run(
                section_agent,
                "生成关于函数定义域值域的笔记",
                context = sc,
            )

In [46]:
result.final_output

'已生成《函数的定义域和值域》笔记。'

In [47]:
print(sc)

SectionContext(definition='函数的定义域和值域是初等数学中的基本概念。\n\n定义域（Domain）指的是一个函数所允许的自变量（输入）的取值范围。换句话说，就是所有可以代入函数表达式中，使其有意义的x的集合。比如函数f(x) = 1/x的定义域就是x ≠ 0的所有实数，因为x=0时分母为零没有意义。\n\n值域（Range）则是函数所有可能输出（函数值）的集合。也就是说，定义域中的每一个自变量取值，经过函数的运算后，所有能得到的y值构成的集合就是函数的值域。例如对于f(x) = x^2，若定义域是所有实数，那么值域就是大于等于0的所有实数。\n\n总结：\n- 定义域：所有允许输入的x值\n- 值域：对应输出的所有y值\n\n接下来会根据以上定义生成相关题型。', examples='好的，以下是一道选择题和一道填空题，涵盖了“函数的定义域和值域”这个主题：\n\n**选择题：**\n\n已知函数 \\( f(x) = \\sqrt{x-2} \\)，下列哪个选项是该函数的定义域？\n\nA. \\( x > 0 \\)  \nB. \\( x \\geq 2 \\)  \nC. \\( x \\leq 2 \\)  \nD. 一切实数\n\n---\n\n**填空题：**\n\n若函数 \\( g(x) = 2x + 1 \\) 的定义域为所有实数，则其值域为\\_\\_\\_\\_\\_\\_\\_\\_。\n\n---\n\n**参考答案：**\n\n选择题答案：B  \n填空题答案：所有实数（或写为 \\((-\\infty, +\\infty)\\)）')
